In [1]:
import os
import tweepy


In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

consumer_key= config.get('KEY','consumer_key')
consumer_secret= config.get('KEY','consumer_secret')
access_token= config.get('KEY','access_token')
access_token_secret= config.get('KEY','access_token_secret')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.get('KEY','json_loc')

auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
!pip install google.cloud --user

Traceback (most recent call last):
  File "/Users/arnavgohil/.conda/envs/RealTimeDP/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "/Users/arnavgohil/.conda/envs/RealTimeDP/lib/python3.8/site-packages/pip/_internal/cli/main.py", line 8, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/Users/arnavgohil/.conda/envs/RealTimeDP/lib/python3.8/site-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/Users/arnavgohil/.conda/envs/RealTimeDP/lib/python3.8/site-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "/Users/arnavgohil/.conda/envs/RealTimeDP/lib/python3.8/site-packages/pip/_internal/cli/cmdoptions.py", line 22, in <module>
    from pip._internal.cli.progress_bars import BAR_TYPES
  File "/Users/arnavgohil/.conda/envs/RealTimeDP/lib/python3.8/site-packages/pip/_intern

In [1]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image = vision.Image()

def parseImage(imageURL):
    image.source.image_uri = image_url
    response = client.text_detection(image=image)
    string = ""
    for text in response.text_annotations:
        string =  text.description
        return string


In [4]:
def getDay(string) :
    s = ""
    for char in string:
        if char.isdigit():
            s += char

    return int(s)

In [5]:
import re
def parseString(string):
    val = dict()

    #Date
    match=(re.search("/ ", string))
    i = int(match.end())
    match=(re.search("\)", string))
    j = int(match.start())
    print(string[i:j])
    val['day'] = getDay(string[i:string.find("2021") - 1])
    val['month'] = 'April'
    val['year'] = '2021'


    #+ve cases
    i = string.find("Positive Cases", string.find("Positive Cases")+1)
    match = (re.search("Tests Conducted", string))
    j = int(match.start())
    val['positive'] = string[i + len("Positive Cases") + 1 : j - 1]

    #Tests
    match=(re.search("Tests Conducted", string))
    i = int(match.end())
    match=(re.search("Positivity Rate", string))
    j = int(match.start())
    val['tests'] = string[i+1:j-1]

    #Recovered
    match=(re.search("%", string))
    i = int(match.end())
    match=(re.search("Deaths", string))
    j = int(match.start())
    val['recovered'] = string[i+1:j-1]

    #Deaths
    match=(re.search("Deaths", string))
    i = int(match.end())
    match=(re.search("COVID-19 Patient Management", string))
    j = int(match.start())
    val['deaths'] = string[i+1:j-1]

    #Tot Vac
    match=(re.search("Beneficiaries vaccinated in last 24 hours", string))
    i = int(match.end())
    match=(re.search("Beneficiaries vaccinated 1", string))
    j = int(match.start())
    val['vaccinated'] = string[i+1:j-1]

    #1st dose
    match=(re.search("dose in last 24 hours", string))
    i = int(match.end())
    match=(re.search("Beneficiaries vaccinated 2", string))
    j = int(match.start())
    val['first_dose'] = string[i+1:j-1]

    #2nd dose
    pat = "dose in last 24 hours"
    pos = string.find(pat)+1
    p1 = string.find(pat, pos+1)
    i = p1 + len(pat)
    match=(re.search("Cumulative beneficiaries vaccinated so far", string))
    j = int(match.start())
    val['second_dose'] = string[i+1:j-1]

    #Active Cases
    match=(re.search("Active Cases", string))
    i = int(match.end())
    match=(re.search("Total Number of Containment Zones as on date", string))
    j = int(match.start())
    val['active_cases'] = string[i+1:j-1]

    #Containment
    match=(re.search("Total Number of Containment Zones as on date: ", string))
    i = int(match.end())
    match=(re.search("Calls received", string))
    j = int(match.start())
    val['zones'] = string[i:j-1]

    print(val)
    return val


In [24]:
#Add data to MySQL

import mysql.connector as connector
conn = connector.connect(
  host="localhost",
  user="root",
  passwd=config.get('KEY','pass'),
  database="casesheet"
)
cursor = conn.cursor()

sql = "INSERT INTO cases VALUES ({},{},{},{},{},{},{},{},{},{},{},{})"

def addData(val):
    try:
        # Executing the SQL command
        cursor.execute(sql.format(val['day'],
                                  val['month'],
                                  val['year'],
                                  val['positive'],
                                  val['tests'],
                                  val['recovered'],
                                  val['deaths'],
                                  val['vaccinated'],
                                  val['first_dose'],
                                  val['second_dose'],
                                  val['active_cases'],
                                  val['zones']
                                  ))
        # Commit your changes in the database
        conn.commit()

    except:
        print("### Error caught ###\n")
        conn.rollback()


In [19]:
date_since = "2021-04-01"
search_words = "Delhi Health Bulletin (from:CMODelhi)"

tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(30)

for tweet in tweets:
    media = tweet.entities.get('media', [])
    image_url = media[0]['media_url']
    parseString(parseImage(image_url))

conn.close()

May 5th 2021
{'day': 5, 'month': 'April', 'year': '2021', 'positive': '20960', 'tests': '79491', 'recovered': '19209', 'deaths': '311', 'vaccinated': '90365', 'first_dose': '64983', 'second_dose': '25382', 'active_cases': '91859', 'zones': '47704'}
May 4th 2021
{'day': 4, 'month': 'April', 'year': '2021', 'positive': '19953', 'tests': '74654', 'recovered': '18788', 'deaths': '338', 'vaccinated': '89297', 'first_dose': '61662', 'second_dose': '', 'active_cases': '90419', 'zones': '46173'}
May 3rd 2021
{'day': 3, 'month': 'April', 'year': '2021', 'positive': '18043', 'tests': '61045', 'recovered': '20293', 'deaths': '448', 'vaccinated': '1611', 'first_dose': '1260', 'second_dose': '351', 'active_cases': '89592', 'zones': '44052'}
May 2nd 2021
{'day': 2, 'month': 'April', 'year': '2021', 'positive': '20394', 'tests': '71997', 'recovered': '24444', 'deaths': '407', 'vaccinated': '49633', 'first_dose': '28775', 'second_dose': '20858', 'active_cases': '92290', 'zones': '42098'}
May 1st 2021
